In [1]:
# import torch
import numpy as np
import json
import os
# import cv2
import random
import glob
# import matplotlib.pyplot as plt
# import seaborn as sns

import pandas as pd

In [2]:
data = json.load(open('/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/ens_nms.json', 'r'))
print(len(data))
fillter_2_class = pd.read_csv('/home/hana/sonnh/kaggle-vin/fold_1_1920_pretrain_all_100_anhdung/bin2class_063.csv')


207175


In [15]:
c = 0
for anno in data:
    if anno['score'] ==0:
        c += 1
print(c)

54994


In [70]:
len(fillter_2_class)

c = 0
for string in fillter_2_class['PredictionString']:
    if '14 0.' in string: 
        c +=1 
#         print(string[string.find('14 0.'):])
print(c)

c = 0
for string in fillter_2_class['PredictionString']:
    if '14 1 0 0 1 1' in string: c += 1
print(c)

3000
0


In [8]:
res = {}
thong_ke = []
x = []
c_small = 0
for anno in data:
#     if anno['score'] < 0.05:
#         continue
#         
    image_id = anno['image_id'][:-6]

    x.append(image_id)
    
    xmin = int(anno['bbox'][0])
    ymin = int(anno['bbox'][1])
    xmax = xmin + int(anno['bbox'][2])
    ymax = ymin + int(anno['bbox'][3])
    
    if int(anno['bbox'][2]) < 5 or int(anno['bbox'][3]) < 5: 
        c_small += 1
        continue
    
    if anno['category_id'] == 12:
        thong_ke.append(anno['score'])
    
    label = [anno['category_id'], anno['score'], xmin, ymin, xmax, ymax]
    if image_id not in res:
        res[image_id] = [label]
    else:
        res[image_id].append(label)

# for anno in data:
#     image_id = anno['image_id'][:-6]
#     if image_id not in res:
#         print(1)
#         res[image_id] = '14 1 0 0 1 1'
        
print(len(res))

3000


In [43]:
c_small

43

In [38]:
res_1 = {}
for image_id in res:
    labels = []
    best = []
    for label in res[image_id]:
        if label[0] == 0:
            if best:
                if best[1] < label[1]:
                    best = label
            else:
                best = label
        else:
            labels.append(label)
    
    if best: labels.append(best)
    res_1[image_id] = labels
res = res_1



In [39]:
res_1 = {}
for image_id in res:
    labels = []
    best = []
    for label in res[image_id]:
        if label[0] == 3:
            if best:
                if best[1] < label[1]:
                    best = label
            else:
                best = label
        else:
            labels.append(label)
    if best: labels.append(best)
    res_1[image_id] = labels
res = res_1



In [9]:
res_final = {}
for image_id in res:
    res_final[image_id] = ''
    for anno in res[image_id]:
        res_final[image_id] += '{} {} {} {} {} {} '.format(anno[0], anno[1], anno[2],anno[3], anno[4],anno[5])
res = res_final

In [58]:
for i in range(3000):
    if fillter_2_class.iloc[i]['image_id'] not in res:

        res[fillter_2_class.iloc[i]['image_id']] = '14 1 0 0 1 1'
len(res)

3000

# 2 class

In [10]:
c = 0
for i in range(3000):
    if fillter_2_class.iloc[i]['PredictionString'] == '14 1 0 0 1 1':
        res[fillter_2_class.iloc[i]['image_id']] = '14 1 0 0 1 1'
    
    if '14 0.' in  fillter_2_class.iloc[i]['PredictionString']:
        string = fillter_2_class.iloc[i]['PredictionString']
        prob = float(string[string.find('14 0.'):].split()[1])
        
        if prob >= 0.95:
            c += 1
            res[fillter_2_class.iloc[i]['image_id']] = string[string.find('14 0.'):]
        else:
            res[fillter_2_class.iloc[i]['image_id']] += ' '
            res[fillter_2_class.iloc[i]['image_id']] += string[string.find('14 0.'):]

In [12]:
len(res)

3000

In [13]:
thong_ke.sort()
thong_ke[::-1]

[0.53564,
 0.40845,
 0.35425,
 0.35181,
 0.28735,
 0.26782,
 0.22754,
 0.19543,
 0.18262,
 0.17285,
 0.16589,
 0.16223,
 0.16162,
 0.13989,
 0.13379,
 0.13135,
 0.13135,
 0.12427,
 0.11853,
 0.09436,
 0.0672,
 0.06696,
 0.06409,
 0.06256,
 0.06064,
 0.05991,
 0.05484,
 0.05426,
 0.05164,
 0.05054,
 0.05029,
 0.04907,
 0.0488,
 0.04559,
 0.04099,
 0.03964,
 0.03635,
 0.03427,
 0.03247,
 0.0321,
 0.03043,
 0.02856,
 0.02753,
 0.02728,
 0.026,
 0.02502,
 0.02478,
 0.02464,
 0.02394,
 0.02338,
 0.02325,
 0.02293,
 0.02222,
 0.02173,
 0.02109,
 0.02083,
 0.02029,
 0.01942,
 0.01933,
 0.01852,
 0.01692,
 0.01685,
 0.01636,
 0.01622,
 0.01617,
 0.01611,
 0.01553,
 0.01542,
 0.01489,
 0.01457,
 0.01432,
 0.01426,
 0.01402,
 0.0139,
 0.01382,
 0.01306,
 0.01304,
 0.0127,
 0.01254,
 0.0123,
 0.01166,
 0.0116,
 0.0115,
 0.01131,
 0.01091,
 0.01088,
 0.01083,
 0.01069,
 0.01068,
 0.01067,
 0.01065,
 0.01049,
 0.01037,
 0.01034,
 0.01031,
 0.0102,
 0.01001,
 0.00996,
 0.00976,
 0.00974,
 0.00972,
 

# save

In [14]:
s = pd.DataFrame({'image_id': res.keys(), 'PredictionString': res.values()})
s.to_csv('/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/ens_nms.csv', index = False)


In [48]:
s.head()

,image_id,PredictionString
0,4112f6a0a3a7509212801f2b3132dfda,14 0.9916724204085768 0 0 1 1
1,0ef99f92d68d16b8912cc073731ffad1,0 0.10632 1197 818 1484 1114 0 0.00299 1363 10...
2,0e580dda54b8d1df7e3a888c41e69704,14 0.9738401125650853 0 0 1 1
3,abbe42351ae1e98ba1bec5733e5dab40,0 0.00293 961 732 1206 990 0 0.00117 1781 1977...
4,fd16e07868605bc0136a2884be2742cb,0 0.43091 1625 702 1962 1088 0 0.05701 1698 99...


In [29]:
s.head()

,image_id,PredictionString
0,785140281117595d1818b8133d43d3bc,3 0.11713 1225 908 1823 1199 11 0.04538 735 12...
1,fa3640d131e70778d70164f476b828d9,0 0.50391 1588 568 1928 924 13 0.49878 781 997...
2,80f3da71b6af9134388dee22f0d386d9,3 0.43628 908 1225 1945 1559 0 0.39771 1271 59...
3,4dcaf539b6497ad02f2734e1b78c9678,3 0.64893 1106 992 2094 1286 0 0.61914 1444 50...
4,04cc236166c1ff638ff6200f2f2a397d,0 0.63623 1255 483 1544 791 3 0.39258 958 1108...


In [55]:
len(s[s['PredictionString'] == '14 1 0 0 1 1'])

1713